In [ ]:
# snnTorch Imports
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen
from snntorch import surrogate
from snntorch import functional as SF

# Torch Imports
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Python Libary Imports
import matplotlib.pyplot as plt
import numpy as np
import itertools
from snntorch import utils
import random
import configparser

# Script Imports
from test_config import getHyperParameters

In [ ]:
#network parameters
nin = 2
nout = 1
rand_range_start = 1
rand_range_end = 10
beta = 0.95
spike_grad = surrogate.fast_sigmoid(slope=25)

In [ ]:
class EONS:
    def __init__(self,params):
        self.params = params
        self.template_network = None
        pass
    def make_template_network(self,nin:int,nout:int):
        hidden = random.randint(rand_range_start,rand_range_end)
        hidden2 = random.randint(rand_range_start,rand_range_end)
        hidden3 = random.randint(rand_range_start,rand_range_end)
        hidden4 = random.randint(rand_range_start,rand_range_end)
        hidden5 = random.randint(rand_range_start,rand_range_end)
        hidden6 = random.randint(rand_range_start,rand_range_end)
        net = nn.Sequential(nn.Linear(nin,hidden),
                            snn.Leaky(beta=beta,spike_grad=spike_grad),
                            nn.Linear(hidden,hidden2),
                            snn.Leaky(beta=beta,spike_grad=spike_grad),
                            nn.Linear(hidden2,hidden3),
                            snn.Leaky(beta=beta,spike_grad=spike_grad),
                            nn.Linear(hidden3,hidden4),
                            snn.Leaky(beta=beta,spike_grad=spike_grad),
                            nn.Linear(hidden4,hidden5),
                            snn.Leaky(beta=beta,spike_grad=spike_grad),
                            nn.Linear(hidden5,hidden6),
                            snn.Leaky(beta=beta,spike_grad=spike_grad),
                            nn.Linear(hidden6,nout),
                            snn.Leaky(beta=beta,spike_grad=spike_grad))
        self.template_network = net
        return None
    
    def add_node(self):
        """
        This method adds a neuron to a randomly selected hidden layer and updates the 
        in_features to the next layer accordingly.
        """
        # Don't do anything if the network didn't initialize properly
        if self.template_network is None:
            print("Template network not initialized")
            return
        
        # Find indicies for all linear layers
        layer_indices = [i for i, layer in enumerate(self.template_network) if isinstance(layer, nn.Linear)]

        # Exclude input and output layers
        if (len(layer_indices) <= 2):
               print("Not enough layers to modify (only input/output layers present)")
               return

        hidden_layer_indices = layer_indices[1:-1] # Only hidden layers
        selected_index = random.choice(hidden_layer_indices) # Select random layer from hidden layer list
        next_index = layer_indices[layer_indices.index(selected_index) + 1] # Get the next layer so that we can update the input

        # Get the layers
        linear = self.template_network[selected_index]
        next_linear = self.template_network[next_index]

        in_feat= linear.in_features # Keeping in features the same
        new_out_feat = linear.out_features + 1 # Adding neuron to the out

        # Creating updated hidden layer
        new_linear = nn.Linear(in_feat, new_out_feat)

        # Ensuring it keeps the same weights and biases
        with torch.no_grad():
             new_linear.weight[:linear.out_features] = linear.weight # Copies weights from old layer to new one
             new_linear.bias[:linear.out_features] = linear.bias # Copies bias value

        # Repeat for the next hidden layer. This ensures that the output of prev layer matches input of next layer
        next_out = next_linear.out_features
        new_next_linear = nn.Linear(new_out_feat, next_out)
        with torch.no_grad():
            new_next_linear.weight[:, :linear.out_features] = next_linear.weight # Copies only the part of the next layer's weight matrix that connects to the existing neurons
            new_next_linear.bias = next_linear.bias # Copies biases for output layer neuron

        # Replacing both layers in the sequential
        layers = list(self.template_network)
        layers[selected_index] = new_linear
        layers[next_index] = new_next_linear
        
        self.template_network = nn.Sequential(*layers)
        print(f"Added a neuron to output of layer {selected_index} and updated input to layer {next_index}")

    def remove_node(self):
        """
        This method removes a neuron from a randomly selected hidden layer.
        The in_features of the next layer is updated accordingly.
        If there isn't enough neurons in the layer to remove (if # of neurons is <= 1),
        the function simply returns and no mutation occurs. 
        """
        # Don't do anything if the network didn't initialize properly
        if self.template_network is None:
            print("Template network not initialized")
            return
        
        # Find indicies for all linear layers
        layer_indices = [i for i, layer in enumerate(self.template_network) if isinstance(layer, nn.Linear)]

        # Exclude input and output layers
        if (len(layer_indices) <= 2):
               print("Not enough layers to modify (only input/output layers present)")
               return

        hidden_layer_indices = layer_indices[1:-1] # Only hidden layers
        selected_index = random.choice(hidden_layer_indices) # Select random layer from hidden layer list
        next_index = layer_indices[layer_indices.index(selected_index) + 1] # Get the next layer so that we can update the input

        # Get the layers
        linear = self.template_network[selected_index]
        next_linear = self.template_network[next_index]

        # Recursively call method until there are more than one neuron in the out features
        if (linear.out_features <= 1):
            print(f"Not enough neurons in layer {selected_index}. Cannot remove neuron.")
            self.remove_node()

        in_feat= linear.in_features # Keeping in features the same
        new_out_feat = linear.out_features - 1 # Removing neuron to the out

        # Creating updated hidden layer
        new_linear = nn.Linear(in_feat, new_out_feat)

        # Ensuring it keeps the same weights and biases
        with torch.no_grad():
             new_linear.weight[:] = linear.weight[:new_out_feat] # Keep the first N-1 neurons' weights
             new_linear.bias[:] = linear.bias[:new_out_feat] # Copies bias values except the one being removed

        # Repeat for the next hidden layer. This ensures that the output of prev layer matches input of next layer
        next_out = next_linear.out_features
        new_next_linear = nn.Linear(new_out_feat, next_out)
        with torch.no_grad():
            new_next_linear.weight[:, :] = next_linear.weight[:, :new_out_feat] # Copies all output rows, but only the first new_out_feat input columns
            new_next_linear.bias = next_linear.bias # Copies biases for output layer neuron

        # Replacing both layers in the sequential
        layers = list(self.template_network)
        layers[selected_index] = new_linear
        layers[next_index] = new_next_linear
        
        self.template_network = nn.Sequential(*layers) # Update network
        print(f"Removed a neuron to output of layer {selected_index} and updated input to layer {next_index}")

    def update_node_param(self):
        """
        This method udpates the weights and bias of one randomly selected neuron in a randomly selected
        hidden layer.

        NOTE: There are a couple of lines that are commented out. These lines were created for visualization purposes
              so that I could see if the neuron parameters were really updating. If you want to see those results,
              simply uncomment those lines and uncomment the necessary lines in the cell below this one. 
        """
        # Don't do anything if the network didn't initialize properly
        if self.template_network is None:
            print("Template network not initialized")
            return
        
        # Find indicies for all linear layers
        layer_indices = [i for i, layer in enumerate(self.template_network) if isinstance(layer, nn.Linear)]

        # Select random layer
        selected_index = random.choice(layer_indices) 
        linear = self.template_network[selected_index]

        # Select a random neuron and generate random weights/bias
        new_weight = torch.rand(1)
        new_bias = torch.rand(1)
        neuron_index = random.choice(range(linear.out_features))

        #print("Newly generated weight and bias")
        #print(new_weight)
        #print(new_bias)
        
        layers = list(self.template_network) # Convert network to list

        #old_weights = layers[selected_index].weight[neuron_index].clone() 
        #old_bias = layers[selected_index].bias[neuron_index].clone() 

        # Update neuron weights and bias
        with torch.no_grad():
            layers[selected_index].weight[neuron_index] = new_weight
            layers[selected_index].bias[neuron_index] = new_bias

        self.template_network = nn.Sequential(*layers) # Updated network

        print(f"Updated neuron {neuron_index} in layer {selected_index}")

        #return selected_index, neuron_index, old_weights, old_bias 

    def add_edge(self):
        """
        Dynamically add a synapse (new connection) between two randomly selected layers in the network.
        This method adds a new nn.Linear layer between two existing layers.
        """
        if self.template_network is None:
            print("Template network not initialized")
            return
        
        # Convert the network to a list of layers
        layers = list(self.template_network)

        # Find indicies for all linear layers
        layer_indices = [i for i, layer in enumerate(self.template_network) if isinstance(layer, nn.Linear)]

        # Exclude input and output layers
        if (len(layer_indices) <= 2):
               print("Not enough layers to modify (only input/output layers present)")
               return

        hidden_layer_indices = layer_indices[1:-1] # Only hidden layers
        
        # Choose two random layers that are next to each other 
        selected_index_1 = random.choice(hidden_layer_indices)  # First layer
        selected_index_2 = selected_index_1 + 2 # Second layer
        
        new_leak = snn.Leaky(beta=beta,spike_grad=spike_grad) # Creating a leaky along with synapses
        
        # Insert the new synapse between the two selected layers (selected_index_1 --> new_synapse --> selected_index_2)
        # Get in_features of second layer and out_features of first layer
        out_features_1 = layers[selected_index_1].out_features 
        in_features_2 = layers[selected_index_2].in_features

        # Create new connection
        new_synapse = nn.Linear(out_features_1, in_features_2)
        
        # Insert new layer between the selected layers
        layers.insert(selected_index_2, new_synapse)
        layers.insert(selected_index_2 + 1, new_leak)

        print(f"Added new synapse at {selected_index_2}.")
        
        # Update the network with the new layers
        self.template_network = nn.Sequential(*layers)

    def remove_edge(self):
        """
        Dynamically remove a synapse (new connection) between two randomly selected layers in the network.
        This method removes a new nn.Linear layer between two existing layers.
        """
        if self.template_network is None:
            print("Template network not initialized")
            return
        
        # Convert the network to a list of layers
        layers = list(self.template_network)

        # Find indicies for all linear layers
        layer_indices = [i for i, layer in enumerate(self.template_network) if isinstance(layer, nn.Linear)]

        # Exclude input and output layers
        if (len(layer_indices) <= 2):
               print("Not enough layers to modify (only input/output layers present)")
               return

        hidden_layer_indices = layer_indices[1:-1] # Only hidden layers
        
        # Choose a random hidden layer to delete
        selected_index_1 = random.choice(hidden_layer_indices)  # Chosen layer
        selected_index_2 = selected_index_1 + 1 # Associated leaky layer
        
        # Remove chosen layer
        remove_layers = [selected_index_1, selected_index_2]
        layers = [layer for index, layer in enumerate(layers) if index not in remove_layers]
        
        # Fix I/O between synapses
        layer_indices = [i for i, layer in enumerate(layers) if isinstance(layer, nn.Linear)]

        # Do not run for last index b/c there are no more synapses to check (index out of range)
        for layer_idx in layer_indices[:-1]:
             # Get the out features of the first layer and see if it matches the in features of the next layer
             out_features = layers[layer_idx].out_features
             in_features = layers[layer_idx+2].in_features

             # Make in features of next layer equal out features of previous layer if they don't match
             if (out_features != in_features):
                modified_layer = nn.Linear(out_features, layers[layer_idx+2].out_features)

                # Copy the existing weights and biases to the new layer
                with torch.no_grad():
                    modified_layer.weight.data = layers[layer_idx + 2].weight.data.clone()
                    modified_layer.bias.data = layers[layer_idx + 2].bias.data.clone()


                layers[layer_idx+2] = modified_layer

        print(f"Removed a synapse at {selected_index_1}.")
        
        # Update the network with the new layers
        self.template_network = nn.Sequential(*layers)

    def update_edge_param(self):
        """
        This method randomly selects a synapse (i.e., a connection between two neurons),
        and updates the weight of that synapse.

        NOTE: The input neuron's parameters aren't directly affected by the synapse update because the input neuron doesn't learn its own weight 
              — it only passes its value to the next layer. Thus, only the weight (synapse) and output neuron's bias are updated.
        
        """
        if self.template_network is None:
            print("Template network not initialized")
            return

        # Convert the network to a list of layers
        layers = list(self.template_network)

        # Find all linear layers in the network
        layer_indices = [i for i, layer in enumerate(layers) if isinstance(layer, nn.Linear)]

        # Randomly select a layer (excluding the input and output layers)
        selected_index = random.choice(layer_indices)
        linear_layer = layers[selected_index]

        # Ensure there are enough input and output neurons before proceeding
        if linear_layer.in_features <= 1 or linear_layer.out_features <= 1:
            print(f"Layer at index {selected_index} does not have enough neurons. Skipping.")
            self.update_edge_param()

        # Select random neuron indices for both the input and output neurons
        input_neuron_index = random.choice(range(linear_layer.in_features))  # Random input neuron
        output_neuron_index = random.choice(range(linear_layer.out_features))  # Random output neuron

        #old_weight_input = linear_layer.weight[output_neuron_index, input_neuron_index].clone()

        #old_weight_output = linear_layer.weight[output_neuron_index, input_neuron_index].clone()
        #old_bias_output = linear_layer.bias[output_neuron_index].clone()

        # Update the synapse weight (i.e., the connection between input neuron and output neuron)
        new_weight = torch.rand(1)  # Random weight for the synapse
        with torch.no_grad():  
            # Accessing the specific weight between the two neurons and updating it
            linear_layer.weight[output_neuron_index, input_neuron_index] = new_weight

        # Update the bias for the both neurons
        new_bias = torch.rand(1)
        with torch.no_grad():  
            linear_layer.bias[output_neuron_index] = new_bias

        #print("Newly generated weight and bias")
        #print(new_weight)
        #print(new_bias)


        # Update the network with the new layers (though layers should already be updated)
        self.template_network = nn.Sequential(*layers)

        print(f"Updated the synapse connecting input neuron {input_neuron_index} to output neuron {output_neuron_index} "
            f"in layer {selected_index}.")
        
        #return selected_index, input_neuron_index, old_weight_input, output_neuron_index, old_weight_output, old_bias_output




In [ ]:
config = configparser.ConfigParser()
config.read('config.ini')
hyperparameters = getHyperParameters(config)

test_eons = EONS(hyperparameters)
test_eons.make_template_network(50, 10)
print(test_eons.template_network, end="\n\n")
test_eons.add_node()
print(test_eons.template_network, end="\n\n")
test_eons.remove_node()
print(test_eons.template_network, end="\n\n")
test_eons.update_node_param()
print("\n")
test_eons.add_edge()
print(test_eons.template_network, end="\n\n")
test_eons.remove_edge()
print(test_eons.template_network, end="\n\n")
test_eons.update_edge_param()
print("\n")

# UNCOMMENT THE LINES BELOW IF YOU WANT TO PRINT OUT RESULTS FROM UPDATE_NODE_PARAM() AND SEE UPDATES
# layer_index, neuron_index, old_weights, old_bias = test_eons.update_node_param() 
# print("\nWeight and Bias before the update")
# print(old_weights)
# print(old_bias)

# print("\nWeight and Bias after the update")
# layers = list(test_eons.template_network)
# print(layers[layer_index].weight[neuron_index])
# print(layers[layer_index].bias[neuron_index])

# UNCOMMENT THE LINES BELOW IF YOU WANT TO PRINT OUT RESULTS FROM UPDATE_EDGE_PARAM() AND SEE UPDATES
# layer_index, input_neuron_index, old_weight_input, output_neuron_index, old_weight_output, old_bias_output = test_eons.update_edge_param()
# print("\nWeight and Bias before the update")
# print("Input Neuron:")
# print(f"\tWeight: {old_weight_input}")
# print("\nOutput Neuron:")
# print(f"\tWeight: {old_weight_output}")
# print(f"\tBias: {old_bias_output}")

# print("\nWeight and Bias after the update")
# layers = list(test_eons.template_network)
# print("Input Neuron:")
# print(f"\tWeight: {layers[layer_index].weight[output_neuron_index, input_neuron_index]}")
# print("\nOutput Neuron:")
# print(f"\tWeight: {layers[layer_index].weight[output_neuron_index, input_neuron_index]}")
# print(f"\tBias: {layers[layer_index].bias[output_neuron_index]}")